In [1]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

from docx2pdf import convert
import os

import pandas as pd
import json

In [2]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [3]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [6]:
fecha = '2023-05-23'

In [7]:
query_insp = {'query': 'select id, canhero from dataset_291595 where date(fecha_registro)=\'{fech}\''.format(fech=fecha)}
project = 'https://app.amigocloud.com/api/v1/projects/31805/sql'
select_insp = amigocloud.get(project, query_insp)
data_insp = select_insp['data']
data_insp

[{'id': 21, 'canhero': '18084 / JUSTINIANO DORA PAZ DE'},
 {'id': 22, 'canhero': '86 / AGUILERA TARADELLES JOSE LUIS'},
 {'id': 23, 'canhero': '4808 / GUTIERREZ SOLIZ VICENTE'}]

In [8]:
for j in data_insp:
    id_insp = j['id']
    print('Generando reporte de:', j['canhero'], '................................................')
    # SELECT DATOS DE LAS INPECCION
    query = {'query': 'select * from dataset_291595 where id={id}'.format(id=id_insp)}
    project = 'https://app.amigocloud.com/api/v1/projects/31805/sql'
    select = amigocloud.get(project, query)
    data = select['data'][0]
    # convertir fecha a formato dd/mm/aaaa
    data['fecha_registro'] = datetime.strptime(data['fecha_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
    # convertir la insp en fecha
    insp = collections.namedtuple("insp", data.keys())(*data.values())
    
    # DATOS DE MUESTRAS
    query = {'query': 'select * from dataset_291604 where calidad_ref_muestra=\'{amigo_id}\''.format(amigo_id=insp.amigo_id)}
    project = 'https://app.amigocloud.com/api/v1/projects/31805/sql'
    select = amigocloud.get(project, query)
    data = select['data']
    # se recorre las muestras para crear un obj y agregarlo a una lista
    muestras =[]
    for i in data:
        # se conviente a entrero
        i['arranquio_de_cepa'] = int(i['arranquio_de_cepa'])
        muestra = collections.namedtuple("muestra", i.keys())(*i.values())
        muestras.append(muestra)
    
    # CALCUALR PROMEDIO DE PESOS
    keys_to_average = ['astillas', 'tocon', 'punta', 'canha_corta', 'canha_larga', 'arranquio_de_cepa', 'estimacion_promedio']
    promedio = {}
    for key in keys_to_average:
        total = sum([item[key] for item in data])
        average = total / len(data)
        promedio[key] = average
    promedio['perdida_total'] = promedio['astillas'] + promedio['tocon'] + promedio['punta'] + promedio['canha_corta'] + promedio['canha_larga']
    promedio['perdida_porcen'] = (promedio['perdida_total'] / promedio['estimacion_promedio'])*100
    promedio['arranquio_de_cepa'] = int(promedio['arranquio_de_cepa'])
    promedio_muestras = collections.namedtuple("promedio", promedio.keys())(*promedio.values())
    
    # SE EXTRAE LOS DATOS DE LAS FOTOS EXISTENTES EN LAS MUESTAS
    fotos = []
    for i in muestras:
        query = {'query': 'select muestra.nombre_muestra, muestra.obs, gal.s3_filename from dataset_291604 muestra inner join gallery_41829 gal on muestra.amigo_id = gal.source_amigo_id where muestra.amigo_id=\'{amigo_id}\''.format(amigo_id=i.amigo_id)}
        project = 'https://app.amigocloud.com/api/v1/projects/31805/sql'
        select = amigocloud.get(project, query)
        if len(select['data'])>0:
            data = select['data'][0]
        else:
            continue
        #data = select['data'][0]
        fotos.append(data)
        print(data)
    
    # SE INSTANCIA AL TEMPLATE
    doc = DocxTemplate(ruta + "/templates/tpl_rpt_calidad-cosecha.docx")
    
    # SE DESCARGAN LAS FOTOS
    lista_fotos_inline = []
    for foto in fotos:
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({
            'foto': docxtpl.InlineImage(doc, image_descriptor=ruta+'/fotos/' + foto['s3_filename'], width=Mm(60)), 
            'muestra': foto['nombre_muestra'],
            'obs':foto['obs']
        })
    
    # SE ESTABLESE LA FIRMA DEL TECNICO
    firma_respon = None
    if insp.responsable == 'Juan Pablo Jaldin':
        firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta+'/templates/firma_jaldin.png', width=Mm(60))
    else:
        firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta+'/templates/firma_rogelio.png', width=Mm(60))
    
    # NORMBRE DEL ARCHIVO
    filename = '/_' + insp.canhero.split(' / ')[0] + '_CDC_' + insp.fecha_registro.replace('/','-') + '_' + insp.canhero.split(' / ')[1] + '_' + str(insp.id) + '.docx'
    # CREAR EL CONTEXTO DEL DOC Y CREAR EL DOC
    context = {'insp':insp, 'muestras':muestras, 'promedio':promedio_muestras, 'fotos':lista_fotos_inline, 'firma':firma_respon}
    doc.render(context)
    doc.save(ruta + '/informes' + filename)
    print('Trarea terminada.')
print('END==============================================')

Generando reporte de: 18084 / JUSTINIANO DORA PAZ DE ................................................
{'nombre_muestra': 'M2', 'obs': 'Poca perdida ', 's3_filename': 'IMG_20230523_093446.jpg'}
Trarea terminada.
Generando reporte de: 86 / AGUILERA TARADELLES JOSE LUIS ................................................
{'nombre_muestra': 'M1', 'obs': 'Tacones ', 's3_filename': '20230523_113635.jpg'}
{'nombre_muestra': 'M2', 'obs': '.', 's3_filename': '20230523_120150.jpg'}
Trarea terminada.
Generando reporte de: 4808 / GUTIERREZ SOLIZ VICENTE ................................................
{'nombre_muestra': 'M1', 'obs': 'Corte de base bueno ', 's3_filename': '41829_20230523123718036.jpg'}
Trarea terminada.
END==============================================


## Convertir a PDF

In [9]:
path = r'' + ruta + '/informes'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 3 INFORMES
3 INFORMES


['D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/18084_CDC_23-05-2023_JUSTINIANO DORA PAZ DE_21.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/4808_CDC_23-05-2023_GUTIERREZ SOLIZ VICENTE_23.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/86_CDC_23-05-2023_AGUILERA TARADELLES JOSE LUIS_22.docx']

In [1]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdfs'))
    pdf = filename.replace('.docx','.pdf').replace('informes','pdfs').split('/')[-1]
    convert(filename, r'D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON\pdf\\' + pdf)
    i+=1
print('END')

NameError: name 'rutas' is not defined

## Cargar a DRIVE

In [11]:
# CONEXION A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1096764895697-57btfmtn7rdejufct5qq4j9mi61j0unl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [12]:
contenido = os.listdir(ruta + r'/pdf')
dic_files = {}
for file in contenido:
    split = file.split('_')
    dic_files[int(split[0])] = file
print(len(dic_files), 'informes')
dic_files

3 informes


{18084: '18084_CDC_23-05-2023_JUSTINIANO DORA PAZ DE_21.pdf',
 4808: '4808_CDC_23-05-2023_GUTIERREZ SOLIZ VICENTE_23.pdf',
 86: '86_CDC_23-05-2023_AGUILERA TARADELLES JOSE LUIS_22.pdf'}

In [13]:
# CARGAR LOS PDFS A GOOGLE DRIVE
dic_urls = {}
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  #print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "1RA_INSPECCION"):
      fileID = file['id']

for file in dic_files:
    file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
    file1['title'] = dic_files[file]
    file1.SetContentFile(ruta + r'/pdf//' + dic_files[file])
    file1.Upload() # Upload the file.
    print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))
    permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    dic_urls[file] = file1['alternateLink']
dic_urls
print('end')

Created file 18084_CDC_23-05-2023_JUSTINIANO DORA PAZ DE_21.pdf with mimeType application/pdf
Created file 4808_CDC_23-05-2023_GUTIERREZ SOLIZ VICENTE_23.pdf with mimeType application/pdf
Created file 86_CDC_23-05-2023_AGUILERA TARADELLES JOSE LUIS_22.pdf with mimeType application/pdf
end
